<strong><b><font size="5">Web Scraping, Data base and SQL </font></b></strong>

## Problem Definition

When extracting data from web sites we are extracting unstructured data, which makes any type of analysis difficult.
In this Lab, we will extract data from a web page, format it in xml and store it in a structured database. Once the data is stored, we will use the SQL Language for data manipulation and analysis.

As our ultimate goal is analysis and not storage, we will create the database in the computer's memory, available only at run time. No data will be stored on disk. This is a fast, efficient and professional way to analyze data extracted from web sites without the need for a storage pipeline.




## Data Source

Our data source will be the website https://www.worldwildlife.org. Browse the page: https://www.worldwildlife.org/species/directory View the source code and understand the data that will be extracted from the 3 columns with information about animals.

## Loading the Used Packages on This Jupyter Notebook

In [1]:
# Python Language Version
from platform import python_version
print('Python Language Version Used In This Jupyter Notebook:', python_version())

Python Language Version Used In This Jupyter Notebook: 3.8.3


In [2]:
# Imports
import sqlite3
import requests
import lxml.html
import numpy as np
import pandas as pd

## Web Scraping

In [3]:
# Creates page request variable
request_web = requests.get("https://www.worldwildlife.org/species/directory")

In [4]:
# Displays the response (200 indicates Ok connection response)
request_web

<Response [200]>

In [5]:
# We will extract the HTML code from the page and convert it to xml for easy manipulation and formatting
html_to_xml = lxml.html.fromstring(request_web.text)

In [6]:
# View
html_to_xml

<Element html at 0x18014ea1f40>

The site has a table, with ** <thead> ** and ** <tbody> **, both with ** <tr> **, and this code will extract all rows from the table at once:

In [7]:
# Extract all lines
trow = html_to_xml.xpath('//tr') 

We check if all lines are the same length, if yes, they are from the same table.

In [8]:
# Checks the length of lines
[len(T) for T in trow[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

It is better to extract the table using the ** <tbody> ** tag, instead of the ** <tr> ** tag, as this way we extract the formatted table.

In [9]:
# Extracts the data from the table
tr = html_to_xml.xpath('//tbody') 

In [11]:
# Element type
tr

[<Element tbody at 0x18014e8d130>]

In [12]:
# We check how many items we have in the table
len(tr[0].getchildren())

50

In [13]:
# or 
tablebody = tr[0]
len(tablebody)

50

In [14]:
# Now let's get the text of 1 item in the table
tablebody.getchildren()[0].text_content()

'\n\t\t\t\t\t\t\tAfrican Elephant\n\t\t\t\t\t\t\tLoxodonta africana\n\t\t\t\t\t\t\tVulnerable\n\t\t\t\t\t\t'

We have 3 terms (3 columns). We can then use the code below to return each column of a table row, as an example.

In [15]:
# Extract columns from a row
tablebody.getchildren()[0].getchildren()

[<Element td at 0x18014edf3b0>,
 <Element td at 0x18014edf400>,
 <Element td at 0x18014edf090>]

In [16]:
# One print for each item
print(tablebody.getchildren()[0].getchildren()[0].text_content())
print(tablebody.getchildren()[0].getchildren()[1].text_content())
print(tablebody.getchildren()[0].getchildren()[2].text_content())

African Elephant
Loxodonta africana
Vulnerable


Let's take a sample of the data.

In [17]:
# Data sample dictionary
data_sample = {"Common name"         :[], 
               "Scientific name"     :[],
               "Conservation status" :[]}

In [19]:
# Loop through the table to extract a sample of data
for row in tablebody.getchildren()[0:3]:
    data_sample["Common name"].append(row.getchildren()[0].text_content())
    data_sample["Scientific name"].append(row.getchildren()[1].text_content())
    data_sample["Conservation status"].append(row.getchildren()[2].text_content())

In [21]:
# View sample data
data_sample

{'Common name': ['African Elephant',
  'African Wild Dog',
  'Albacore Tuna',
  'African Elephant',
  'African Wild Dog',
  'Albacore Tuna'],
 'Scientific name': ['Loxodonta africana',
  'Lycaon pictus',
  'Thunnus alalunga',
  'Loxodonta africana',
  'Lycaon pictus',
  'Thunnus alalunga'],
 'Conservation status': ['Vulnerable',
  'Endangered',
  'Near Threatened',
  'Vulnerable',
  'Endangered',
  'Near Threatened']}

In [22]:
# We convert the data sample (dictionary) to Pandas dataframe
pd.DataFrame(data_sample)

,Common name,Scientific name,Conservation status
0,African Elephant,Loxodonta africana,Vulnerable
1,African Wild Dog,Lycaon pictus,Endangered
2,Albacore Tuna,Thunnus alalunga,Near Threatened
3,African Elephant,Loxodonta africana,Vulnerable
4,African Wild Dog,Lycaon pictus,Endangered
5,Albacore Tuna,Thunnus alalunga,Near Threatened


We now store all the data in the Python dictionary.

In [23]:
# Dictionary to receive all data
data = {"Common name"         :[],
        "Scientific name"     :[],
        "Conservation status" :[]}

In [24]:
# Loop to extract all data from the table
for row in tablebody.getchildren():
    data["Common name"].append(row.getchildren()[0].text_content().strip())
    data["Scientific name"].append(row.getchildren()[1].text_content().strip())
    data["Conservation status"].append(row.getchildren()[2].text_content().strip())      

In [26]:
# Convert to pandas dataframe and view
pd.DataFrame(data).head(10)

,Common name,Scientific name,Conservation status
0,African Elephant,Loxodonta africana,Vulnerable
1,African Wild Dog,Lycaon pictus,Endangered
2,Albacore Tuna,Thunnus alalunga,Near Threatened
3,Amazon River Dolphin,Scientific Name Inia geoffrensis,
4,Amur Leopard,Panthera pardus orientalis,Critically Endangered
5,Arctic Fox,Vulpes lagopus,Least Concern
6,Arctic Wolf,Canis lupus arctos,Least Concern
7,Asian Elephant,Elephas maximus indicus,Endangered
8,Beluga,Delphinapterus leucas,Near Threatened
9,Bigeye Tuna,Thunnus obesus,Vulnerable


In [29]:
# We created the dataframe
df = pd.DataFrame(data)   

In [30]:
# We replace missing values with NaN
df.replace('', np.nan, inplace = True)

In [31]:
# Iteration through items and screen printing
for i in df.itertuples():
    print(i[1:])

('African Elephant', 'Loxodonta africana', 'Vulnerable')
('African Wild Dog', 'Lycaon pictus', 'Endangered')
('Albacore Tuna', 'Thunnus alalunga', 'Near Threatened')
('Amazon River Dolphin', 'Scientific Name Inia geoffrensis', nan)
('Amur Leopard', 'Panthera pardus orientalis', 'Critically Endangered')
('Arctic Fox', 'Vulpes  lagopus', 'Least Concern')
('Arctic Wolf', 'Canis lupus arctos', 'Least Concern')
('Asian Elephant', 'Elephas maximus indicus', 'Endangered')
('Beluga', 'Delphinapterus leucas', 'Near Threatened')
('Bigeye Tuna', 'Thunnus obesus', 'Vulnerable')
('Black Rhino', 'Diceros bicornis', 'Critically Endangered')
('Black Spider Monkey', 'Ateles paniscus', 'Vulnerable')
('Black-footed Ferret', 'Mustela nigripes', 'Endangered')
('Blue Whale', 'Balaenoptera musculus', 'Endangered')
('Bluefin Tuna', 'Thunnus Thynnus', 'Endangered')
('Bonobo', 'Pan paniscus', 'Endangered')
('Bornean Elephant', 'Elephas maximus borneensis', 'Endangered')
('Bornean Orangutan', 'Pongo pygmaeus', '


## Storage in Database and SQL Language

To use the sqlite3 module, you must first create a connection object that represents the database and, optionally, create a cursor object, which will assist you in executing all SQL statements.

More in: https://www.sqlite.org/index.html

In [32]:
# Creates connection to the sqlite database that will be created only in memory (does not save the database to disk)
conn = sqlite3.connect(":memory:")

In [33]:
# Create the cursor
cursor = conn.cursor()

In [34]:
# Creates the sql statement for creating a table in the database
sql1 = """ CREATE TABLE SPECIES(Common_name PRIMARY KEY NOT NULL, Scientific_name, Conservation_status) """

We use NOT NULL for the restrictions of not allowing NULL in the Common_name column records. A primary key column cannot have NULL values. The keyword PRIMARY in a relational database indicates that each row in the column is unique.

In [36]:
# Executes the SQL statement
cursor.execute(sql1)

In [37]:
# Loop through data extracted from the web and insertion in the database table
for rec in df.itertuples():
    
    # Instrução SQL de Insert
    insert_table = """ INSERT INTO SPECIES(Common_name, Scientific_name, Conservation_status) VALUES (?,?,?) """
    
    # Executa a instução
    cursor.execute(insert_table, rec[1:5])

In [38]:
# The commit writes the data to the database
conn.commit()

Vamos consultar os dados armazenados na tabela em memória.

In [39]:
# Create query to select the data in the database table
query1 = """ SELECT * FROM SPECIES """

In [49]:
# Run the query
result1 = cursor.execute(query1)

In [50]:
# Loop for returning the query and printing the data
for row in result1:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  African Elephant
Scientific_name =  Loxodonta africana
Conservation_status =  Vulnerable 

Common_name =  African Wild Dog
Scientific_name =  Lycaon pictus
Conservation_status =  Endangered 

Common_name =  Albacore Tuna
Scientific_name =  Thunnus alalunga
Conservation_status =  Near Threatened 

Common_name =  Amazon River Dolphin
Scientific_name =  Scientific Name Inia geoffrensis
Conservation_status =  None 

Common_name =  Amur Leopard
Scientific_name =  Panthera pardus orientalis
Conservation_status =  Critically Endangered 

Common_name =  Arctic Fox
Scientific_name =  Vulpes  lagopus
Conservation_status =  Least Concern 

Common_name =  Arctic Wolf
Scientific_name =  Canis lupus arctos
Conservation_status =  Least Concern 

Common_name =  Asian Elephant
Scientific_name =  Elephas maximus indicus
Conservation_status =  Endangered 

Common_name =  Beluga
Scientific_name =  Delphinapterus leucas
Conservation_status =  Near Threatened 

Common_name =  Bigeye Tuna
Scie

We will now return specific data.

In [42]:
# Create query to select the data in the database table
# Pay attention to the use of quotation marks

query2 = """ SELECT * FROM SPECIES WHERE Conservation_status = "Least Concern" """

In [51]:
# Run the query
result2 = cursor.execute(query2)

In [52]:
# Loop for returning the query and printing the data
for row in result2:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Arctic Fox
Scientific_name =  Vulpes  lagopus
Conservation_status =  Least Concern 

Common_name =  Arctic Wolf
Scientific_name =  Canis lupus arctos
Conservation_status =  Least Concern 

Common_name =  Bowhead Whale
Scientific_name =  Balaena mysticetus
Conservation_status =  Least Concern 

Common_name =  Brown Bear
Scientific_name =  Ursus arctos
Conservation_status =  Least Concern 

Common_name =  Common Bottlenose Dolphin
Scientific_name =  Tursiops truncates
Conservation_status =  Least Concern 



Updating table values ​​in the database using Primary Key as a reference.

Update, Insert and Delete statements are DML (Data Manipulation Language) statements in SQL language.

In [45]:
# Create query to update the data in the database table

dml_1 = """ UPDATE SPECIES SET Conservation_status = "Least-Concern" WHERE Common_name = "Gray Whale" """

In [46]:
# Run the Query
cursor.execute(dml_1)
conn.commit()

In [47]:
# Create query to select the data in the database table

query3 = """ SELECT * FROM SPECIES WHERE Conservation_status = "Least-Concern" """

In [54]:
# Run the Query
result3 = cursor.execute(query3)

In [55]:
# Loop for returning the query and printing the data
for row in result3:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Gray Whale
Scientific_name =  Eschrichtius robustus
Conservation_status =  Least-Concern 



Inserting data into the table in the database.

In [56]:
# Create query to insert the data into the database table

dml_2 = " INSERT INTO SPECIES(Common_name, Scientific_name, Conservation_status) \
          VALUES ('Cat', 'Felis catus', 'Important' ) "

In [57]:
# Run the Query
cursor.execute(dml_2)
conn.commit()

In [62]:
# Create query to select the data in the database table
query4 = """ SELECT * FROM SPECIES WHERE Conservation_status = "Important" """

In [63]:
# Run the Query
result4 = cursor.execute(query4)

In [64]:
# Loop for returning the query and printing the data
for row in result4:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Cat
Scientific_name =  Felis catus
Conservation_status =  Important 



Excluding the same data entered above.

In [65]:
# Create query to delete data in the database table

dml_3 = """ DELETE from SPECIES where Common_name = "Cat" """

In [66]:
# Run the Query
cursor.execute(dml_3)
conn.commit()

In [67]:
# Create query to select the data in the database table
query5 = """ SELECT * FROM SPECIES WHERE Common_name = "Cat" """

In [68]:
# Run the Query
result5 = cursor.execute(query5)

In [69]:
# Loop pelo retorno da query e impressão dos dados
for row in result5:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

The database does not return any value, as it has no values ​​that we request.

In [70]:
# We closed the connection to the database
conn.close()